In [2]:
import alpaca_trade_api as tradeapi
import alpacakeys as keys
import json
import requests
from PortfolioOpt import PortfolioOpt
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import time
import yfinance as yf

In [3]:
api = tradeapi.REST(keys.api_key, keys.secret_key, base_url='https://paper-api.alpaca.markets')

In [4]:
def get_pos(api):
    rawpos = api.list_positions()
    pos = {}
    for i in rawpos:
        pos[i.symbol] = float(i.qty)


    return pos


def get_quote(ticker):
    close = yf.download(ticker, start=date.today()).iloc[-1]["Close"]
    return close


def calc_new_shares(port_val, tickers, weights):

    new_shares = {}

    dol_inv = weights * port_val
    for ticker, inv in zip(tickers, dol_inv):
        last_price = get_quote(ticker)
        new_shares[ticker] = np.floor(inv/last_price)

    return new_shares

def get_pos_diff(current, new):

    diff = {key: new[key] - current.get(key, 0) for key in new.keys()}

    sell_all = {}
    for ticker, shares in current.items():
        if ticker not in diff:
            sell_all[ticker] = -shares

    diff.update(sell_all)
    return diff

def place_order(pos_dict):

    sell_dict = {k:np.abs(v) for k,v in pos_dict.items() if v < 0}
    buy_dict = {k:v for k,v in pos_dict.items() if v > 0}


    for ticker, shares in sell_dict.items():
        try:
            print(f"Sell: {ticker}, {shares}")
            api.submit_order(
                symbol=ticker,
                side='sell',
                type='market',
                qty=str(shares),
                time_in_force='day'
            )
        except:
            pass

    for ticker, shares in buy_dict.items():
        try:
            print(f"Buy: {ticker}, {shares}")
            api.submit_order(
                symbol=ticker,
                side='buy',
                type='market',
                qty=str(shares),
                time_in_force='day'
            )
        except:
            pass

In [7]:
tickers = ["RCL", "AAL", "MSFT", "BAC", "SNAP", "AMZN", "KO", "DIS", "COST", "VZ", "AMD", "NVDA", "WMT", "V", "HD", "DPZ", "JBLU", "MDLZ",
           "TSLA", "WEN", "UPS", "PLUG"]

# tickers = ["RCL", "AAL", "MSFT"]


start_date = date.today() - relativedelta(years=3)
opt = PortfolioOpt(tickers, start=str(start_date), lookahead=0)
weights = opt.optimize_portfolio(opt_for="sharpe", print_results=False)['x'].round(4)

In [8]:
weights

array([0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.3282, 0.    , 0.2154, 0.    , 0.    , 0.    , 0.    , 0.143 ,
       0.    , 0.    , 0.1542, 0.    , 0.    , 0.1592])

In [10]:
port_val = float(api.get_account().equity)
port_val

106181.17

In [11]:
current = get_pos(api)
current

{'MSFT': 495.0}

In [12]:
new = calc_new_shares(port_val, tickers, weights)
new

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'RCL': 0.0,
 'AAL': 0.0,
 'MSFT': 0.0,
 'BAC': 0.0,
 'SNAP': 0.0,
 'AMZN': 0.0,
 'KO': 0.0,
 'DIS': 0.0,
 'COST': 93.0,
 'VZ': 0.0,
 'AMD': 254.0,
 'NVDA': 0.0,
 'WMT': 0.0,
 'V': 0.0,
 'HD': 0.0,
 'DPZ': 39.0,
 'JBLU': 0.0,
 'MDLZ': 0.0,
 'TSLA': 27.0,
 'WEN': 0.0,
 'UPS': 0.0,
 'PLUG': 632.0}

In [13]:
order_pos = get_pos_diff(current, new)
order_pos

{'RCL': 0.0,
 'AAL': 0.0,
 'MSFT': -495.0,
 'BAC': 0.0,
 'SNAP': 0.0,
 'AMZN': 0.0,
 'KO': 0.0,
 'DIS': 0.0,
 'COST': 93.0,
 'VZ': 0.0,
 'AMD': 254.0,
 'NVDA': 0.0,
 'WMT': 0.0,
 'V': 0.0,
 'HD': 0.0,
 'DPZ': 39.0,
 'JBLU': 0.0,
 'MDLZ': 0.0,
 'TSLA': 27.0,
 'WEN': 0.0,
 'UPS': 0.0,
 'PLUG': 632.0}

In [14]:
place_order(order_pos)

Sell: MSFT, 495.0
Buy: COST, 93.0
Buy: AMD, 254.0
Buy: DPZ, 39.0
Buy: TSLA, 27.0
Buy: PLUG, 632.0
